## Author: Igor Dzierwa
## Laboratory 3 - Implementation of Gradient Descent in neural networks with one hidden layer

In this lab we are going to implement a neural network with one hidden layer. The architecture of our neural network is as follows:

- Input layer with 3 neurons
- One hidden layer with 2 neurons
- Output layer with 1 neuron

The dataset has 200 data each one has three features selected from a random normal distribution with a mean of zero and the standard deviation of 5 and the actual labels 0 or 1.

Weights are also initialized randomly from normal distribution with *Xavier* initialization technique. Hidden  layer activation is *Tanh* and the output layer activation is *Sigmoid* with *binary cross-entropy* as loss.

Also, for this task use batch gradient descent algorithm.

Please define different functions for each part of the training process:
- initialization
- activation functions and their derivative 
- loss function and its derivative 
- forward propagation
- backpropagation 
- parameters update
- training

#### Libraries import

In [6]:
import numpy as np
import random

#### Data initialization

In [20]:
def dataset_initialization(mean=0, sd=5, count=200):
    sample_normal_dist = np.random.normal(mu, sigma, 1000)
    
    features_list = [random.choices(sample_normal_dist, k=3) for i in range(count)]
    values_list = [1 if sum(i) >= 0 else 0 for i in features_list]
    
    return features_list, values_list

dataset = dataset_initialization()

#### Sigmoid and Tanh activation functions

In [2]:
def sigmoid(self, x):
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(self, x):
    return self.sigmoid(x) * (1 - self.sigmoid(x))

def tanh(z):
    ez = np.exp(z)
    enz = np.exp(-z)
    return (ez - enz)/ (ez + enz)

def tanh_deriv(z):
    a = tanh(z)
    return 1 - a**2

#### Cross entropy loss function

In [ ]:
def error_formula(y, output):
    return -y * np.log(output) - (1-y) * np.log(1-output)

def error_term_frumula(x, yhat, y):
    return (yhat - y)*x